## What is this?

This notebook aims to complete the "Modeling and Hedging Livestock Risk Protection" assignment given by Ash Capital Management as a final interview.

Modules used:
* Numpy
* Pandas
* Matplotlib
* Seaborn
* Scipy
* ipywidgets

## Instructions to use

* Install Python version 3.13.5
* [Clone](https://github.com/Tres300/Interview-Assignment) the repository.
* Create a virtual enviroment.
* Inside the repository, run ``` pip install -r requirements.txt ```.
* Run all cells in the notebook to regenerate contents and make the interactable charts work.
* Enjoy!



## Import libraries

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy.stats import norm
from ipywidgets import interact, widgets
sns.set_theme(style='whitegrid')

## Chart data generation functions

In [20]:
# Constants (shouldn't need to be modified)
FIRST_BACKSTOP_ACTIVATION_PERCENTAGE  = 1.5     # Must be < SECOND_BACKSTOP_ACTIVATION_PERCENTAGE
FIRST_BACKSTOP_USDA_LOSS_PERCENTAGE   = 0.9     # Must be <= 1
SECOND_BACKSTOP_ACTIVATION_PERCENTAGE = 5.0     # Must be > FIRST_BACKSTOP_ACTIVATION_PERCENTAGE
SECOND_BACKSTOP_USDA_LOSS_PERCENTAGE  = 1.0     # Must be <= 1
RESOLUTION = 2

# Black‑Scholes delta of a European put (N(d1)).
# This functions assumes r (the Risk-free rate) is constant 0.
def DeltaPrice(S: float, K: float, T: float, sigma: float) -> float:
  # Avoid division by 0
  if T <= 0 or sigma == 0 or S == 0 or K == 0:
    # Derivative of max(K ‑ S, 0)
    if S < K:
      return -1.0
    else:
      return 0.0

  d1 = (np.log(S / K) + (sigma ** 2) * 0.5) / (sigma * np.sqrt(T))
  return norm.cdf(d1) - 1

# Black‑Scholes price of a European put.
# This functions assumes r (the Risk-free rate) is constant 0.
def PutPrice(S: float, K: float, T: float, sigma: float) -> float:
  # Avoid division by 0
  if T == 0 or sigma == 0 or S == 0 or K == 0:
    # Simplifies
    return max(K - S, 0.0)

  d1 = (np.log(S / K) + (sigma ** 2) * T * 0.5) / (sigma * np.sqrt(T))
  d2 = d1 - (sigma * np.sqrt(T))
  price = K * norm.cdf(-d2) - S * norm.cdf(-d1)
  return price

# Payoff function that includes backstops if enabled. This is done by adding the
# remaining loss percentage covered by the USDA times the PutPrice when the
# strike_price is k1/k2 respectively, to the original loss (premium minus the
# normal PutPrice).
def Payoff(underlying_price: float,
           strike_price: float,
           premium: float,
           fraction_of_year: float,
           volatility: float,
           enable_backstop_one: bool,
           enable_backstop_two: bool,
           first_backstop_k1: float,
           second_backstop_k2: float) -> float:

  # Get the payoff without backstops
  result = premium - PutPrice(underlying_price, strike_price, fraction_of_year, volatility)

  # Add the PutPrice covered by the USDA at the first backstop, effectively
  # removing it since the payoff is negative.
  if enable_backstop_one:
    result += FIRST_BACKSTOP_USDA_LOSS_PERCENTAGE * PutPrice(underlying_price, first_backstop_k1, fraction_of_year, volatility)

  # Add the PutPrice covered by the USDA at the second backstop, effectively
  # removing it since the payoff is negative. If the first backstop is enabled,
  # add the remaining 10% loss to remove it (since the USDA now cover 100%).
  if enable_backstop_two:
    backstop_two_loss_percentage = SECOND_BACKSTOP_USDA_LOSS_PERCENTAGE
    if enable_backstop_one:
      backstop_two_loss_percentage = SECOND_BACKSTOP_USDA_LOSS_PERCENTAGE - FIRST_BACKSTOP_USDA_LOSS_PERCENTAGE
    result += backstop_two_loss_percentage * PutPrice(underlying_price, second_backstop_k2, fraction_of_year, volatility)

  return result

# Delta has the same parameter list as Payoff, and can be used to find the delta
# function that (including backstops if enabled). This is done by taking the
# opposite of the delta without backstops, and adding the delta when the
# strike_price is k1/k2 respectively times the remaining loss percentage covered
# by the USDA.
def Delta(underlying_price: float,
          strike_price: float,
          premium: float,
          fraction_of_year: float,
          volatility: float,
          enable_backstop_one: bool,
          enable_backstop_two: bool,
          first_backstop_k1: float,
          second_backstop_k2: float) -> float:

  # Get the delta without backstops
  result = -DeltaPrice(underlying_price, strike_price, fraction_of_year, volatility)

  # Add the Delta covered by the USDA at the first backstop, effectively
  # removing it since the delta without backstops is negative.
  if enable_backstop_one:
    result += FIRST_BACKSTOP_USDA_LOSS_PERCENTAGE * DeltaPrice(underlying_price, first_backstop_k1, fraction_of_year, volatility)

  # Add the Delta covered by the USDA at the second backstop, effectively
  # removing it since the delta without backstops negative. If the first
  # backstop is enabled, add the remaining 10% to remove it (since the USDA now
  # cover 100%).
  if enable_backstop_two:
    backstop_two_loss_percentage = SECOND_BACKSTOP_USDA_LOSS_PERCENTAGE
    if enable_backstop_one:
      backstop_two_loss_percentage = SECOND_BACKSTOP_USDA_LOSS_PERCENTAGE - FIRST_BACKSTOP_USDA_LOSS_PERCENTAGE
    result += backstop_two_loss_percentage * DeltaPrice(underlying_price, second_backstop_k2, fraction_of_year, volatility)

  return result


## Chart generating function

In [21]:
# Generates a chart from a bunch of parameters and a payoff_function, used to
# generate different 'y' data (Payoff vs Delta). The payoff_function must have
# the same parameters as Payoff.
def GenerateChart(title: str,
                  x_label: str,
                  y_label: str,
                  payoff_function,
                  strike_price: float,
                  premium: float,
                  fraction_of_year: float,
                  volatility: float,
                  enable_backstop_one: bool,
                  enable_backstop_two: bool,
                  y_range: int,
                  number_of_x_axis_ticks: int,
                  number_of_y_axis_ticks: int):
  # Generate a linspace/array for all x/underlying_price values
  max_x_underlying_price = int(np.ceil(2 * strike_price))
  underlying_price_array = np.linspace(0, max_x_underlying_price, max_x_underlying_price * RESOLUTION)

  # Calculate k1 and k2
  first_backstop_k1  = max(strike_price - (FIRST_BACKSTOP_ACTIVATION_PERCENTAGE  * premium), 0)
  second_backstop_k2 = max(strike_price - (SECOND_BACKSTOP_ACTIVATION_PERCENTAGE * premium), 0)

  # Generate the data array (as pairs of x,y values) using the payoff function
  data_pair_array = [(x, payoff_function(x,
                                         strike_price,
                                         premium,
                                         fraction_of_year,
                                         volatility,
                                         enable_backstop_one,
                                         enable_backstop_two,
                                         first_backstop_k1,
                                         second_backstop_k2))
                     for x in underlying_price_array]

  # Turn the data array into a dataframe for use by seaborn
  df = pd.DataFrame(data_pair_array, columns=["x_values", "y_values"])

  # Create the figure and axis using a mix of seaborn and matplotlib
  fig, ax = plt.subplots(figsize=(10, 7))
  chart = sns.lineplot(data=df, x="x_values", y="y_values",linewidth=2, ax=ax)

  # Set title and axis labels
  ax.set(
    title=title,
    xlabel=x_label,
    ylabel=y_label
  )
  ax.title.set_size(20)
  ax.xaxis.label.set_size(16)
  ax.yaxis.label.set_size(16)

  # Explicity define the number of ticks on each axix
  ax.tick_params(axis="both", which="major", labelsize=12)
  ax.xaxis.set_major_locator(mticker.MaxNLocator(number_of_x_axis_ticks))
  ax.yaxis.set_major_locator(mticker.MaxNLocator(number_of_y_axis_ticks))

  # Highlight y = 0
  ax.axhline(0, color="black", linewidth=1, linestyle="--")

  # Draw x = K
  ax.axvline(strike_price, color="gray", linewidth=1, linestyle="--")

  if enable_backstop_one:
    # Draw x = K1
    ax.axvline(first_backstop_k1, color="gray", linewidth=1, linestyle="--")
  if enable_backstop_two:
    # Draw x = K2
    ax.axvline(second_backstop_k2, color="gray", linewidth=1, linestyle="--")

  # Set the domain and range displayed on the chart
  chart.set(xlim=(0, max_x_underlying_price), ylim=(-(y_range // 2), (y_range // 2)))
  plt.show()

## Interactable chart

In [22]:
# Helper function to be used by GenerateInteractableChart() to set titles,
# labels, and for these charts to be modifyable by having interact call this
# function.
def GenerateInteractableChartHelper(strike_price: float,
                                    premium: float,
                                    fraction_of_year: float,
                                    volatility: float,
                                    enable_backstop_one: bool,
                                    enable_backstop_two: bool,
                                    show_black_scholes_delta: bool,
                                    y_range: int):
  # Generate the LRP Payoff chart. This is a good place to change
  # GenerateChart's parameters.
  title = "LRP Payoff"
  x_label="Underlying Price (S)"
  y_label="Payoff"
  payoff_function = Payoff
  number_of_payoff_ticks = 18
  number_of_price_ticks = 10
  number_of_x_axis_ticks = 10
  number_of_y_axis_ticks = 18

  GenerateChart(title,
                x_label,
                y_label,
                payoff_function,
                strike_price,
                premium,
                fraction_of_year,
                volatility,
                enable_backstop_one,
                enable_backstop_two,
                y_range,
                number_of_x_axis_ticks,
                number_of_y_axis_ticks)

  # Generate the LRP Delta chart if it's enabled. This is also a good place to
  # change the chart's parameters.
  if show_black_scholes_delta:
    title = "LRP Delta"
    x_label="Underlying Price (S)"
    y_label="Delta"
    payoff_function = Delta
    number_of_payoff_ticks = 18
    number_of_price_ticks = 10
    number_of_x_axis_ticks = 10
    number_of_y_axis_ticks = 18

    GenerateChart(title,
                  x_label,
                  y_label,
                  payoff_function,
                  strike_price,
                  premium,
                  fraction_of_year,
                  volatility,
                  enable_backstop_one,
                  enable_backstop_two,
                  y_range,
                  number_of_x_axis_ticks,
                  number_of_y_axis_ticks)

# Function that contains interaction for the LRP Payoff and Delta charts
def GenerateInteractableChart():
  # Change these values to change the default/max/min/step of the sliders
  interact(GenerateInteractableChartHelper,
           strike_price             = widgets.FloatSlider(value=100, min=1,  max=1000, step=0.1),
           premium                  = widgets.FloatSlider(value=10,  min=0,  max=200,  step=0.1),
           fraction_of_year         = widgets.FloatSlider(value=0,   min=0,  max=1,    step=0.01),
           volatility               = widgets.FloatSlider(value=1,   min=0,  max=5,  step=0.1),
           y_range                  = widgets.IntSlider(  value=40,  min=2,  max=1000, step=1),
           enable_backstop_one      = widgets.Checkbox(value=True),
           enable_backstop_two      = widgets.Checkbox(value=True),
           show_black_scholes_delta = widgets.Checkbox(value=False))

GenerateInteractableChart()

interactive(children=(FloatSlider(value=100.0, description='strike_price', max=1000.0, min=1.0), FloatSlider(v…